In [114]:
import psycopg2
import os 
import numpy as np

pw = os.getenv("PSQLPW")
conn = psycopg2.connect("host=localhost dbname=game_recs user=jcbraun password="+pw)

c = conn.cursor()
c.execute('SELECT id FROM recs_player')
users = []
for row in c.fetchall():
  users.append(row[0])
c.execute('SELECT game_id, name FROM recs_game')
games = []
named_games = []
for row in c.fetchall():
  games.append(row[0])
  named_games.append(row[1])
c.execute('SELECT play_time, game_id, player_id FROM recs_activity')
matrix = [[0] * len(users) for game in games]

for row in c.fetchall():
  matrix[games.index(row[1])][users.index(row[2])] = row[0]

In [97]:
u,s,v=np.linalg.svd(matrix, full_matrices=False)


In [113]:
def rank_one_mod(user_vector, game_vector, matrix):
  #m = U'a
  m = np.dot(u.transpose(), a)
  #pVec = a - Um
  p_vec = np.subtract(a, np.multiply(U, m))
  #p = sqrt(p'p)
  p = sqrt(np.multiply(pVec, pVec));
  #P = pVec/p
  P = np.divide(p_vec, p)
  #n = V'b
  n = np.multiply(Vt, b)
  #qVec = b - Vn
  q_vec = np.subtract(b, np.multiply(v, n))
  #q = sqrt(p'p)
  q = sqrt(np.multiply(qVec, qVec))
  #Q = qVec/q
  Q = np.divide(qVec, q)
  rhs = np.multiply(np.appendToVector(m, p), np.appendToVector(n, q))
  expanded_matrix = [[0] + row for row in matrix] + [[0] * len(matrix[0])]
  rhs = np.add(expanded_matrix, rhs);
  u,s,v=np.linalg.svd(expanded_matrix, full_matrices=False)

In [102]:
conn.rollback()

In [107]:
def commit_recs():
    rec_matrix = [[0] * len(games) for user in users]
    for i in range(len(users)):
        for j in range(len(games)):
            propensity = np.dot(z[i], x[j])
            conn = psycopg2.connect("host=localhost dbname=game_recs user=jcbraun password="+pw)
            c = conn.cursor()
            c.execute("INSERT INTO recs_rec (player_id, game_id, score) VALUES ('{0}', '{1}', '{2}')".format(users[i], games[j], propensity))
            rec_matrix[i][j]= propensity
            conn.commit()

KeyboardInterrupt: 